In [1]:
import sys, torch, gym
print(sys.executable)     # should point into .../envs/cooperative/bin/python
#print(marllib.__version__)   # 1.0.3
print(torch.__version__)     # 1.9.0 (or whatever you installed)
print(gym.__version__)

/home/tristongrayston/anaconda3/envs/cooperative/bin/python
1.9.0+cu102
0.22.0


In [6]:
import gym
import time
from gym.envs.registration import register
import argparse
import MARLlib

In [7]:
register(
            id='multigrid-collect-v0',
            entry_point='gym_multigrid.envs:CollectGame4HEnv10x10N2',
        )
env = gym.make('multigrid-collect-v0')

/home/tristongrayston/anaconda3/envs/cooperative/lib/python3.8/site-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment multigrid-collect-v0
  logger.warn(f"Overriding environment {id}")


In [23]:
from MARLlib.marllib import marl
from ray import tune
import ray

env = marl.make_env(environment_name="mpe", map_name="simple_spread", force_coop=True)

mappo = marl.algos.mappo(hyperparam_source="test", lr=tune.grid_search([0.0005, 0.001]))

model = marl.build_model(env, mappo, {"core_arch": "mlp", "encode_layer": tune.grid_search(["8-16", "16-32"])})

+--------------------+----------------+----------------------------------------------------------------------+----------------------------------------------+-------------------------------------+
| Env_Name           | Check_Status   | Error_Log                                                            | Config_File_Location                         | Env_File_Location                   |
+====================+================+======================================================================+==============================================+=====================================+
| gymnasium_mamujoco | Error          | No module named 'gymnasium_robotics'                                 | envs/base_env/config/gymnasium_mamujoco.yaml | envs/base_env/gymnasium_mamujoco.py |
+--------------------+----------------+----------------------------------------------------------------------+----------------------------------------------+-------------------------------------+
| mpe               

In [26]:
mappo.fit(env, model, stop={"timesteps_total": 1000000}, checkpoint_freq=100, share_policy="group")

ValueError: Attempting to start raylet with 1 GPUs, but CUDA_VISIBLE_DEVICES contains [].

In [25]:
ray.shutdown()

In [ ]:
# A start and maybe a fun gif
from ray.tune.registry import register_env
from ray.rllib.env import PettingZooEnv
from gym_multigrid.envs.soccer_game import SoccerGame4HEnv10x15N2
from pettingzoo.utils import from_parallel as parallel_to_aec


In [17]:

def multigrid_soccer_env(config):
    raw_env = SoccerGame4HEnv10x15N2()         # > implements the Parallel API
    aec_env = parallel_to_aec(raw_env)         # > AEC that Ray understands
    return PettingZooEnv(aec_env) 

register_env("multigrid_soccer", multigrid_soccer_env)

env = marl.make_env(
        environment_name="pettingzoo",     # generic PettingZoo adapter
        map_name="multigrid_soccer",       # the Ray string we just registered
        force_coop=True                    # optional: flip to full cooperation
      )


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/grays/OneDrive/Desktop/github things/CooperativeBehaviour/MARLlib/marllib/marl/../envs/base_env/config/pettingzoo.yaml'

In [16]:
dir(parallel_to_aec)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [3]:
_ = env.reset()
nb_agents = len(env.agents)

for _ in range(1):
    #env.render(highlight=True)
    time.sleep(0.1)
    
    ac = [env.action_space.sample() for _ in range(nb_agents)]
    #ac = [0] * nb_agents
    
    obs, _, done, _ = env.step(ac)
    print("Num Agents = ", len(obs))
    print("Obs shape = ", obs[0].T.shape)
    
    if done:
        break

NameError: name 'env' is not defined

In [45]:
# Agent
import numpy as np
import argparse
import gym
from mappo.onpolicy.algorithms.r_mappo.r_mappo import R_MAPPO
from mappo.onpolicy.algorithms.r_mappo.algorithm.rMAPPOPolicy import R_MAPPOPolicy as policy
from mappo.onpolicy.config import get_config

In [46]:
''' rMAPPOPolicy wants a centralized observation space as well. In this case, what we could do is output the whole grid, but that might
cause some unnessessary strife here. It's probably easier in this case to just throw all of the observations into one array.'''

obs_shape = obs[0].T.shape
obs_shape = gym.spaces.Box(0, 32*32, obs_shape)

action_space = gym.spaces.Discrete(3) # One hot encoded action space

cent_obs_space = np.array([4, 6, 7, 7])
cent_obs_space = gym.spaces.Box(0, 32*32, cent_obs_space)

# Gets the default arguments for the policy in question
parser = get_config()                        # build the ArgumentParser
args   = parser.parse_args([])               # empty list ⇒ use every default
#print(vars(args))

p = policy(args, obs_space = obs_shape, cent_obs_space = cent_obs_space, act_space = action_space)
agent = R_MAPPO(args, policy)